In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
import streamlit as st
import geopandas as gpd
from streamlit_folium import folium_static
from sklearn.ensemble import RandomForestRegressor
import requests
import osmnx as ox
import networkx as nx

In [ ]:
df_points = pd.read_csv("GPS Trajectory/go_track_trackspoints.csv")
df_tracks = pd.read_csv("GPS Trajectory/go_track_tracks.csv")


In [ ]:
df_merged = df_points.merge(df_tracks, left_on="track_id", right_on="id", how="left")

In [ ]:
df_final = df_merged[["time_x", "latitude", "longitude", "speed", "track_id"]].copy()
df_final.rename(columns={"time_x": "timestamp", "track_id": "road_id"}, inplace=True)


In [ ]:
df_final["timestamp"] = pd.to_datetime(df_final["timestamp"])

In [ ]:
df_final["hour"] = df_final["timestamp"].dt.hour

In [ ]:
peak_hours = df_final.groupby("hour")["speed"].mean().reset_index()

In [ ]:
X = df_final[["hour"]]
y = df_final["speed"]
model = RandomForestRegressor()
model.fit(X, y)

def predict_speed(hour):
    return model.predict([[hour]])[0]

In [ ]:
gdf = gpd.GeoDataFrame(df_final, geometry=gpd.points_from_xy(df_final.longitude, df_final.latitude))

In [ ]:
st.title("Traffic Congestion Analysis")
st.write("Explore traffic congestion patterns and predictions.")

In [ ]:
st.subheader("Peak Traffic Hours")
fig, ax = plt.subplots()
sns.lineplot(data=peak_hours, x="hour", y="speed", marker="o", ax=ax)
ax.set_title("Average Speed by Hour")
ax.set_xlabel("Hour of the Day")
ax.set_ylabel("Average Speed (km/h)")
st.pyplot(fig)

In [ ]:
st.subheader("Predict Traffic Congestion")
hour_input = st.slider("Select an hour", 0, 23, 12)
predicted_speed = predict_speed(hour_input)
st.write(f"Predicted average speed at {hour_input}:00 is {predicted_speed:.2f} km/h")


In [ ]:
bottleneck_threshold = df_final["speed"].quantile(0.25)
bottleneck_zones = df_final[df_final["speed"] <= bottleneck_threshold]

In [ ]:
m = folium.Map(location=[df_final["latitude"].mean(), df_final["longitude"].mean()], zoom_start=12)
heat_data = list(zip(bottleneck_zones["latitude"], bottleneck_zones["longitude"], bottleneck_zones["speed"]))
HeatMap(heat_data).add_to(m)

In [ ]:
st.subheader("Traffic Congestion Heatmap")
folium_static(m)


In [ ]:

# Recommendations
st.subheader("Recommendations for Traffic Optimization")
st.write("1. Implement adaptive traffic signal control at identified bottleneck zones.")
st.write("2. Encourage alternate routes during peak traffic hours.")
st.write("3. Optimize public transport schedules based on congestion patterns.")
st.write("4. Introduce congestion pricing for high-traffic areas to reduce peak demand.")
